In [3]:
import numpy as np
import h5py

num_steps = 120

examples_file = "../MetatoneQuartetExamples-" + str(num_steps) + "steps" + ".h5"

with h5py.File(examples_file, 'r') as data_file:
        dataset = data_file['examples'][:]
        validation_set = data_file['validation'][:]

In [42]:
# dataset file has examples of shape (2, num_steps), 
#where the first row is the input, and second 
# is the correct output sequence.
# First step is to adapt it into many-to-one format
X = dataset[:,0,:] # take all the input sequences
y = dataset[:,1,-1] # just select the last one of each output sequence
y = y.reshape((-1,1)) # reshape to (n,1)
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (343482, 120)
y shape: (343482, 1)


In [12]:
import keras

Using TensorFlow backend.


In [44]:
num_layers = 1
batch_size = 64
num_units = 32
num_steps = 120

num_input_performers = 4
num_output_performers = 3

vocabulary_size = 9 # len(GESTURE_CODES)
num_input_classes = vocabulary_size ** num_input_performers
num_output_classes = vocabulary_size ** num_output_performers

training_model = keras.models.Sequential()
training_model.add(keras.layers.Embedding(num_input_classes, num_units, input_length=num_steps))
for n in range(num_layers - 1):
    training_model.add(keras.layers.LSTM(num_units, return_sequences=True))
training_model.add(keras.layers.LSTM(num_units))
training_model.add(keras.layers.Dense(num_output_classes, activation='softmax'))
# model.add(Activation('softmax'))

training_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam')
training_model.summary()

# Notes:
# Difficulty of this task is learning from a relatively large input space:
print("Number of input classes:", num_input_classes)
print("Number of output classes", num_output_classes)
# It's handy to use an Embedding layer so that we can learn from integer
# inputs (not one-hot)
# This means that for lower 'num_units', the parameters used for the input 
# embedding outnumber the LSTM layers.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 120, 32)           209952    
_________________________________________________________________
lstm_23 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_12 (Dense)             (None, 729)               24057     
Total params: 242,329
Trainable params: 242,329
Non-trainable params: 0
_________________________________________________________________
Number of input classes: 6561
Number of output classes 729


In [45]:
training_model.fit(X, y, batch_size=batch_size, validation_split=0.15)

Train on 291959 samples, validate on 51523 samples
Epoch 1/1
 75328/291959 [======>.......................] - ETA: 5:12 - loss: 5.1939

KeyboardInterrupt: 